In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
from collections import Counter
from scipy.interpolate import splprep, splev

## Measure feather in pixels and cm^2

In [ ]:
image_path = "example.jpg"
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

feather_contour = max(contours, key=cv2.contourArea)

x, y, feather_width, feather_height = cv2.boundingRect(feather_contour)

total_feather_length_cm = 10.0  # Actual total length of the feather in cm
conversion_factor = total_feather_length_cm / feather_height  # Pixels to cm

output_image = image.copy()
cv2.drawContours(output_image, [feather_contour], -1, (0, 255, 0), 2)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
plt.title("Feather Contour with Grid Overlay")

cm_spacing = 1 
pixel_spacing = cm_spacing / conversion_factor  

num_horizontal_lines = int(total_feather_length_cm // cm_spacing) + 1
num_vertical_lines = int(feather_width * conversion_factor // cm_spacing) + 1

for i in range(0, num_horizontal_lines):
    plt.axhline(y=y + i * pixel_spacing, color='red', linestyle='--', linewidth=0.5)

for j in range(0, num_vertical_lines):
    plt.axvline(x=x + j * pixel_spacing, color='red', linestyle='--', linewidth=0.5)

y_ticks = np.arange(0, num_horizontal_lines) * cm_spacing
x_ticks = np.arange(0, num_vertical_lines) * cm_spacing
plt.yticks(ticks=np.arange(0, num_horizontal_lines) * pixel_spacing + y, labels=y_ticks)
plt.xticks(ticks=np.arange(0, num_vertical_lines) * pixel_spacing + x, labels=x_ticks)

plt.axis('on')
plt.show()

pixel_area = cv2.contourArea(feather_contour)

vane_area_cm2 = pixel_area * (conversion_factor ** 2)

print(f"Estimated area of the feather vane: {vane_area_cm2:.2f} square cm")


## Feather wear measurement

In [ ]:
def load_image(path):
    return cv2.imread(path)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def detect_edges(gray_image):
    return cv2.Canny(gray_image, 50, 150)

def find_contours(edges):
    return cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

def smooth_contours(contours):
    smoothed_contours = []
    for contour in contours:
        epsilon = 0.00001 * cv2.arcLength(contour, True)  
        approx = cv2.approxPolyDP(contour, epsilon, True)
#       hull = cv2.convexHull(approx)  
        smoothed_contours.append(approx)
    return smoothed_contours

def apply_kmeans(image, k=3):
    pixels = image.reshape((-1, 3))
    pixels = np.float32(pixels)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    centers = np.uint8(centers)
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)
    
    return segmented_image, labels.reshape(image.shape[:2]), centers

def identify_clusters(labels, centers):
    counts = Counter(labels.flatten())
    sorted_counts = counts.most_common()
    
    background_label = sorted_counts[0][0]
    feather_label = sorted_counts[1][0]

    return background_label, feather_label

def create_masks(image, labels, background_label, feather_label):
    height = image.shape[0]
    
    background_mask = (labels == background_label).astype(np.uint8) * 255
    feather_mask = (labels == feather_label).astype(np.uint8) * 255
    
    feather_mask[int(4*height/5):, :] = 0
    
    return background_mask, feather_mask

def calculate_wear(image, feather_mask, background_mask):
    contours, _ = find_contours(feather_mask)
    smoothed_contours = smooth_contours(contours)
    
    total_contour_area = 0
    covered_area = np.zeros_like(feather_mask)  
    
    contour_image = np.zeros_like(feather_mask)
    contour_mask = np.zeros_like(feather_mask)
    
    for contour in smoothed_contours:
        contour_area = cv2.contourArea(contour)
        total_contour_area += contour_area
        
        cv2.drawContours(contour_image, [contour], -1, 255, thickness=cv2.FILLED)  
        
        cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)
        
        covered_area +=  (feather_mask == 255)
    
    covered_area_sum = np.sum(feather_mask == 255)  #
    missing_area = total_contour_area - covered_area_sum
    wear_score = missing_area / total_contour_area if total_contour_area != 0 else 0
    wear_mask = np.logical_and(feather_mask == 0, contour_area == 255).astype(np.uint8) * 255
    display_image = image.copy()
    cv2.putText(display_image, f"Total Contour Area: {total_contour_area:.2f}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(display_image, f"Missing Area: {missing_area:.2f}", (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    display_image_rgb = cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(display_image_rgb)
    plt.title('Feather Wear Detection with Annotations')
    plt.axis('off')
    plt.show()
    
    return wear_score, contour_image, wear_mask

def visualize(images, titles):
    plt.figure(figsize=(20, 10))
    for i, (image, title) in enumerate(zip(images, titles)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.show()
    

"""def process_images():
    wear_scores = []
    
    for i in range(1, 42):
        image_path = f'{i}.jpg'
        image = load_image(image_path)
        
        if image is None:
            print(f"Image {image_path} could not be loaded.")
            continue
        
        gray_image = convert_to_grayscale(image)
        edges = detect_edges(gray_image)
        contours, _ = find_contours(edges)
        smoothed_contours = smooth_contours(contours)
        segmented_image, labels, centers = apply_kmeans(image, k=3)
        background_label, feather_label = identify_clusters(labels, centers)
        background_mask, feather_mask = create_masks(image, labels, background_label, feather_label)
        
        wear_score, _, _ = calculate_wear(image, feather_mask, background_mask)
        wear_scores.append({'Feather ID': i, 'Wear Score': wear_score})
    
    df = pd.DataFrame(wear_scores)
    
    return df

wear_scores_df = process_images()
wear_scores_df""" # This was to save all feather wear measurements into one dataframe to save time


def main(image_path):
    image = load_image(image_path)
    gray_image = convert_to_grayscale(image)
    edges = detect_edges(gray_image)
    contours, _ = find_contours(edges)
    smoothed_contours = smooth_contours(contours)
    segmented_image, labels, centers = apply_kmeans(image, k=3)
    background_label, feather_label = identify_clusters(labels, centers)
    background_mask, feather_mask = create_masks(image, labels, background_label, feather_label)
    
    wear_score, contour_image, wear_mask = calculate_wear(image, feather_mask, background_mask)
    
    print("Wear Score:", wear_score)
     
    wear_detected_image = image.copy()
    cv2.drawContours(wear_detected_image, smoothed_contours, -1, (0, 255, 0), 2)

    wear_detected_image_rgb = cv2.cvtColor(wear_detected_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(wear_detected_image_rgb)
    plt.title('Feather Outline and Wear Detection')
    plt.axis('off')
    plt.show()
    
    visualize([
        cv2.cvtColor(image, cv2.COLOR_BGR2RGB),  # Original Image with Wear Overlay
        gray_image,                                        # Grayscale Image
        edges,                                            # Edge-detected Image
        contour_image,                                     # Contour Image
        feather_mask,                                      # Feather Mask
       wear_mask                                          # Wear Mask
    ], [
        'Original Image',
        'Grayscale Image',
        'Edge-detected Image',
        'Contour Image',
        'Feather Mask',
       'Wear Mask'
    ])
